# Full Transfer Functions

In [1]:
%load_ext autoreload
import sys
sys.path.append("..")

In [2]:
%autoreload

import matplotlib
matplotlib.rc_file('matplotlibrc')
import matplotlib.pyplot as plt

%matplotlib inline

matplotlib.rcParams['figure.figsize'] = [10,10]


from astropy.io import fits

import numpy as np
import copy
import pickle

import darkhistory.physics as phys
import darkhistory.utilities as utils
import darkhistory.spec.spectools as spectools
import darkhistory.spec.transferfunction as tf
import darkhistory.spec.transferfunclist as tflist
from darkhistory.spec.spectrum import Spectrum
from darkhistory.spec.spectra import Spectra
import darkhistory.history.tla as tla

from scipy.interpolate import interp1d

from tqdm import tqdm_notebook as tqdm

from darkhistory.electrons.ics.ics_spectrum import ics_spec
from darkhistory.electrons.ics.ics_engloss_spectrum import engloss_spec
from darkhistory.electrons.ics.ics_cooling import get_ics_cooling_tf

import darkhistory.low_energy.lowE_photons as lowE_photons
import darkhistory.low_energy.lowE_electrons as lowE_electrons

# np.set_printoptions(threshold=np.nan)

## Photon Transfer Functions

In [3]:
user = 'hongwan'

highengphot_tflist_arr_raw = pickle.load(open("/Users/"+user+"/Dropbox (MIT)/Photon Deposition/tfunclist_photspec_60eV_complete_coarse.raw", "rb"))
lowengphot_tflist_arr_raw  = pickle.load(open("/Users/"+user+"/Dropbox (MIT)/Photon Deposition/tfunclist_lowengphotspec_60eV_complete_coarse.raw", "rb"))
lowengelec_tflist_arr_raw  = pickle.load(open("/Users/"+user+"/Dropbox (MIT)/Photon Deposition/tfunclist_lowengelecspec_60eV_complete_coarse.raw", "rb"))
CMB_engloss_raw = pickle.load(open("/Users/"+user+"/Dropbox (MIT)/Photon Deposition/CMB_engloss_60eV_complete_coarse.raw", "rb"))

In [4]:
highengphot_tflist_arr_raw[0].rs

array([2634.79908808, 2314.05541151, 2032.35702934, 1784.95081585,
       1567.66226062, 1376.82503156, 1209.21911254, 1062.01647166,
        932.73334369,  819.18832112,  719.46554715,  631.88238918,
        554.9610476 ,  487.40362071,  428.07020512,  375.95966203,
        330.19272489,  289.99716348,  254.69475396,  223.68983515,
        196.45925789,  172.54355785,  151.53920296,  133.09178459,
        116.89003755,  102.66058812,   90.16334132,   79.18742983,
         69.54765596,   61.08136684,   53.64570988,   47.11522249,
         41.37971508,   36.34241186,   31.91831787,   28.03278494,
         24.62025207,   21.62313923,   18.99087584,   16.67904745,
         14.64864634,   12.86541334,   11.29925978,    9.92375978,
          8.71570441,    7.65471001,    6.72287432,    5.90447437,
          5.18570123,    4.55442696,    4.        ])

In [5]:
xes = 0.5 + 0.5*np.tanh([-5., -4.1, -3.2, -2.3, -1.4, -0.5, 0.4, 1.3, 2.2, 3.1, 4])


In [6]:
tf_interp = tflist.TransferFuncInterp(xes, highengphot_tflist_arr_raw)
print(tf_interp.rs)

[   4.            4.55442696    5.18570123    5.90447437    6.72287432
    7.65471001    8.71570441    9.92375978   11.29925978   12.86541334
   14.64864634   16.67904745   18.99087584   21.62313923   24.62025207
   28.03278494   31.91831787   36.34241186   41.37971508   47.11522249
   53.64570988   61.08136684   69.54765596   79.18742983   90.16334132
  102.66058812  116.89003755  133.09178459  151.53920296  172.54355785
  196.45925789  223.68983515  254.69475396  289.99716348  330.19272489
  375.95966203  428.07020512  487.40362071  554.9610476   631.88238918
  719.46554715  819.18832112  932.73334369 1062.01647166 1209.21911254
 1376.82503156 1567.66226062 1784.95081585 2032.35702934 2314.05541151
 2634.79908808]


In [7]:
%%prun
for i in np.arange(100):
    print(i)
    tf_interp.get_tf(2000-i, 0.5)
# print(tf_interp.rs)
# tf_interp.get_tf(200,0.5)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
 

In [8]:
print(highengphot_tflist_arr_raw[0].grid_vals.shape)
a = np.atleast_3d(
                np.stack([tf.grid_vals for tf in highengphot_tflist_arr_raw[0]])
            )
print(a.shape)
print(highengphot_tflist_arr_raw[0][0])
print(highengphot_tflist_arr_raw[0][0].grid_vals.shape)

(51, 325, 500)
(51, 325, 500)
(325, 500)


In [9]:
b = highengphot_tflist_arr_raw[0]
print(b.grid_vals.shape)
b.transpose()
print(b.grid_vals.shape)

(51, 325, 500)
(325, 51, 500)


In [10]:
test1 = Spectrum(np.array([1,2]), np.array([3,4]), spec_type = 'N')
test2 = Spectrum(np.array([1,2]), np.array([3,4]), spec_type = 'N')
a = tf.TransFuncAtRedshift([test1, test2])

In [11]:
a.grid_vals

array([[3, 4],
       [3, 4]])

In [12]:
b = tf.TransFuncAtRedshift(np.array([[3,4,5], [3,4,5]]), eng = np.array([1,2,3]), in_eng = np.array([1, 2]))